In [19]:
from scipy.integrate import *
from pandas import *
from numpy import *
from pylab import *
from scipy import *

In [20]:
# iimport data
d2004 = read_excel("ke_2008_data.xlsx", sheetname = "t2004")
d2005 = read_excel("ke_2008_data.xlsx", sheetname = "t2005")
T4,M4,D4,Z4 =array(d2004['T']),array(d2004['M']),array(d2004['E']),array(d2004['Z'])
T5,M5,D5,Z5 =array(d2005['T']),array(d2005['M']),array(d2005['E']),array(d2005['Z'])

f1,ax1 = subplots(1,2,figsize=[9,5])
f2,ax2 = subplots(2,2,figsize=[12,12])
ax2 = ax2.flatten()

TypeError: read_excel() got an unexpected keyword argument `sheetname`

In [21]:
# time array
ndays,delt = 360*5,1.0/24.0
times = linspace(0,ndays,ndays/delt)

In [22]:
#params
Sr = 1.1
mumD = 2.0
mumM = 0.8
KsD = mumD/0.1
KsM = mumM/0.1
gmaxD = 5.0
gmaxM = 5.0
KgD = gmaxD/0.25
KgM = gmaxM/0.25
epsD = 0.25
epsM = 0.25
phiM = 8/1.0
phiD = 8/1.0
delD = 0.1
delM = 0.1
delV = 0.1
delP = 0.1
delZ = 0.1
betaV = 0.01
betaP = 0.01

params = r_[[Sr,mumD,mumM,KsD,KsM,KgD,KgM,gmaxD,gmaxM,epsD,epsM,phiM,phiD,delD,delM,delV,delP,betaV,betaP]]

inits = r_[[10.0,1e+0,1e+0,1e-1,1e-1,1e+0]]

In [23]:
def bloom(u,t):
    n,m = 1,2
    N,D,M,V,P,Z = u[0],u[1],u[2],u[3],u[4],u[5]
    sfac = sin((t-100)/180.0*3.142)*0.5+0.5
    gamma = 1.0
    dNdt = Sr - sfac*mumD*N/(KsD+N)*D  - sfac*mumM*N/(KsM+N)*M - 0.1*N
    dDdt = sfac*mumD*N/(KsD+N)*D - phiD*D*V - (gmaxD*D**m/(KgD**m+D**m)*Z) - (delD*D)**n
    dMdt = sfac*mumM*N/(KsM+N)*M - phiM*M*P*gamma - (gmaxM*M**m/(KgM**m+M**m)*Z) - (delM*M)**n
    dVdt = betaV*phiD*D*V - (delV*V)**n
    dPdt = betaP*phiM*M*P*gamma - (delP*P)**n
    dZdt = epsD*(gmaxD*D**m/(KgD*m+D**m)*Z) + epsM*(gmaxM*M**m/(KgM**m+M**m)*Z) - (delZ*Z)**n
    return concatenate([r_[[dNdt]],r_[[dDdt]],r_[[dMdt]],r_[[dVdt]],r_[[dPdt]],r_[[dZdt]]])

In [24]:
def bloom_switch(u,t):
    n,m = 1,2
    N,D,M,V,P,Z = u[0],u[1],u[2],u[3],u[4],u[5]
    sfac = sin((t-100)/180.0*3.142)*0.5+0.5
    if ((mod(t,360)>140) and (mod(t,360)<240.0)):
        gamma = 0.0
    else:
        gamma = 1.0
    dNdt = Sr - sfac*mumD*N/(KsD+N)*D  - sfac*mumM*N/(KsM+N)*M - 0.1*N
    dDdt = sfac*mumD*N/(KsD+N)*D - phiD*D*V - (gmaxD*D**m/(KgD**m+D**m)*Z) - (delD*D)**n
    dMdt = sfac*mumM*N/(KsM+N)*M - phiM*M*P*gamma - (gmaxM*M**m/(KgM**m+M**m)*Z) - (delM*M)**n
    dVdt = betaV*phiD*D*V - (delV*V)**n
    dPdt = betaP*phiM*M*P*gamma - (delP*P)**n
    dZdt = epsD*(gmaxD*D**m/(KgD*m+D**m)*Z) + epsM*(gmaxM*M**m/(KgM**m+M**m)*Z) - (delZ*Z)**n
    return concatenate([r_[[dNdt]],r_[[dDdt]],r_[[dMdt]],r_[[dVdt]],r_[[dPdt]],r_[[dZdt]]])

In [25]:
labs = ['Nutrient','Eukaryotic algae','Microcystis','Viruses of Eukaryotes','Cyanophage','Microzooplankton']
cols = ['C0','C1','C2','C3','C4','C5']

ax2t1 = ax2[0].twinx()
ax2t2 = ax2[1].twinx()

u = odeint(bloom,inits,times).T

w = 360*4

for (tseries,lab,c) in zip(u[0:],labs[0:],cols[0:]):
    ax1[0].plot(times-w,tseries,lw=2.0,label=lab,c=c)

for i in r_[[1,2,3,4,5]]:
    if i in [1,2,5]:
        ax2[0].plot(times-w,u[i],lw=2.0,label=labs[i],c=cols[i])
    else:
        ax2t1.plot(times-w,u[i],lw=2.0,label=labs[i],c=cols[i])

u = odeint(bloom_switch,inits,times).T

for (tseries,lab,c) in zip(u[0:],labs[0:],cols[0:]):
    ax1[1].plot(times-w,tseries,lw=2.0,label=lab,c=c)

for i in r_[[1,2,3,4,5]]:
    if i in [1,2,5]:
        ax2[1].plot(times-w,u[i],lw=2.0,label=labs[i],c=cols[i])
    else:
        ax2t2.plot(times-w,u[i],lw=2.0,label=labs[i],c=cols[i])

la = ax1[0].legend()
la.draw_frame(False)
la = ax2t2.legend()
la.draw_frame(False)


NameError: name 'ax2' is not defined

In [26]:
ax2[2].plot(T4,D4,'-o',label=labs[1],c=cols[1])
ax2[2].plot(T4,M4,'-o',label=labs[2],c=cols[2])
ax2[2].plot(T4,Z4,'-o',label=labs[5],c=cols[5])
ax2[3].plot(T5,D5,'-o',label=labs[1],c=cols[1])
ax2[3].plot(T5,M5,'-o',label=labs[2],c=cols[2])
ax2[3].plot(T5,Z5,'-o',label=labs[5],c=cols[5])

ax2[0].set_xlim([amin(T4),amax(T4)+100])
ax2[1].set_xlim([amin(T5),amax(T5)+100])
#ax1.set_xlim([0,360])

l1 = ax2[0].legend()
l2 = ax2[3].legend()
l1.draw_frame(False)
l2.draw_frame(False)

ax1[0].set_xlabel('Time (julian day)')
ax1[0].set_ylabel(r'Biomass (mg L$^{-1}$)')
ax1[1].set_xlabel('Time (julian day)')

ax2[2].set_xlabel('Time (julian day)')
ax2[0].set_ylabel(r'Biomass (mg L$^{-1}$)')
ax2[3].set_xlabel('Time (julian day)')
ax2[2].set_ylabel(r'Biomass (mg L$^{-1}$)')
ax2t1.set_ylabel(r'Biomass (mg L$^{-1}$)')
ax2t2.set_ylabel(r'Biomass (mg L$^{-1}$)')

ax2[0].set_title(r'$Without$ seasonal switching')
ax2[1].set_title(r'$With$ seasonal switching')
ax2[2].set_title('Lake Taihu 2004')
ax2[3].set_title('Lake Taihu 2005')

ax1[0].semilogy()
ax1[1].semilogy()

ax2[0].text(0.07,0.9,'a',ha='center',va='center',color='k',transform=ax2[0].transAxes)
ax2[1].text(0.07,0.9,'b',ha='center',va='center',color='k',transform=ax2[1].transAxes)
ax2[2].text(0.07,0.9,'c',ha='center',va='center',color='k',transform=ax2[2].transAxes)
ax2[3].text(0.07,0.9,'d',ha='center',va='center',color='k',transform=ax2[3].transAxes)

f2.savefig('bloom_succession_virus_grazer')

show()

NameError: name 'ax2' is not defined